In [1]:
### Installing the required packages if not already installed
packages = ['numpy', 'pandas', 'warnings', 'sqlite3', 'sqlite3']

for package in packages:
    try:
        __import__(package)
    except ImportError:
        %pip install {package}

import numpy as np    # For numerical computing
import pandas as pd   # For data manipulation
import sqlite3        # For connecting to SQL database
import os

### Ignoring the warnings
import warnings
warnings.filterwarnings('ignore')

### Pandas display options
pd.options.display.float_format = '{:.4f}'.format

### Setting working directory
os.chdir('/Users/emilwilliamhansen/Desktop/Master-Thesis/Code')

In [2]:
### Extracting the data from the database
conn = sqlite3.connect('Data/data.db')

### Reading the data from the database
stock_ret_d = pd.read_sql('SELECT * FROM filtered_daily_returns', conn).set_index('Date')
stock_ret_m = pd.read_sql('SELECT * FROM filtered_monthly_returns', conn).set_index('Date')
factor_ret_d = pd.read_sql('SELECT * FROM factors_daily', conn).set_index('Date')
factor_ret_m = pd.read_sql('SELECT * FROM factors_monthly', conn).set_index('Date')

### Closing the connection
conn.close()

### Making sure the index is a datetime index
stock_ret_d.index = pd.to_datetime(stock_ret_d.index)
stock_ret_m.index = pd.to_datetime(stock_ret_m.index)
factor_ret_d.index = pd.to_datetime(factor_ret_d.index)
factor_ret_m.index = pd.to_datetime(factor_ret_m.index)

### Extracting risk-free rate
rf_d = factor_ret_d['rf']
rf_m = factor_ret_m['rf']

### Extracting the market return
exc_mkt_ret_d = factor_ret_d['VW']
exc_mkt_ret_m = factor_ret_m['VW']

exc_ew_ret_d = factor_ret_d['EW']
exc_ew_ret_m = factor_ret_m['EW']

### Calculating the excess returns
exc_stock_ret_d = stock_ret_d.sub(rf_d, axis=0)
exc_stock_ret_m = stock_ret_m.sub(rf_m, axis=0)

# Recreating BAB

Code inspired by:
- https://github.com/WenqiAngieWu/Betting-Against-Beta/blob/master/main.py

I'll compute betas on a daily basis to capture as much information as possible. For each beta, denoted as $\beta_t$, we'll only use data from before time $t$. This ensures that each beta is applied within its corresponding time period, avoiding any look-ahead bias.

To do this, we introduce `closed='left'` in the `rolling` function. This sets the window to include the past $w$ observations while excluding the current one, essentially replicating the effect of shifting the betas but in a more robust manner.


In [3]:
### Getting the 3-day log returns
log_stock_ret_d = np.log(1 + exc_stock_ret_d.astype(float))
log_market_ret_d = np.log(1 + exc_mkt_ret_d)

three_day_exc_stock_ret = log_stock_ret_d.rolling(window=3).sum()
three_day_exc_mkt_ret = log_market_ret_d.rolling(window=3).sum()

### Getting the volatility
# we follow the original paper and use a 1 year (252 days) window.
# we also use daily returns, and not the three-day returns.
# we also restrict the window to at least have 120 days of data.
vol_stock_ret_d = log_stock_ret_d.rolling(window=252, min_periods=120, closed='left').std()
vol_mkt_ret_d = log_market_ret_d.rolling(window=252, min_periods=120, closed='left').std()

### Getting the correlations
# we follow the original paper and use a 5 year (252*5 days) window.
# we also use the three-day returns, and not the daily returns.
# we also restrict the window to at least have 750 days of data.
correlation = three_day_exc_stock_ret.rolling(window=252*5, min_periods=750, closed='left').corr(three_day_exc_mkt_ret)

### Getting the beta
# we compute the beta using the formula from the original paper.
# we shrink the betas using the formula from the original paper.
beta_d = 0.6 * correlation * vol_stock_ret_d.div(vol_mkt_ret_d, axis=0) + 0.4 * 1

### Getting the monthly beta
# Since the beta is just based on past values, the first beta of each month is the beta we are interested in.
beta_m = beta_d.resample('M').first()

In [4]:
### Getting the beta ranks
betarank_m = beta_m.rank(axis=1, method='average')

### Getting the median
betamedian_m = betarank_m.median(axis=1)

### Normalizing the betas
k_m = 2 / abs(betarank_m.sub(betamedian_m, axis=0)).sum(axis=1)

### Getting the weights
w_m = betarank_m.sub(betamedian_m, axis=0).mul(k_m, axis=0)
wH_m = w_m.applymap(lambda x:x if x > 0 else 0)
wL_m = w_m.applymap(lambda x:-x if x < 0 else 0)

### Getting the portfolio returns
exc_ret_bab_H_m = (stock_ret_m.mul(wH_m, axis=1).sum(axis=1) - factor_ret_m['rf']) / (beta_m.mul(wH_m, axis=1).sum(axis=1).replace(0, np.nan))
exc_ret_bab_L_m = (stock_ret_m.mul(wL_m, axis=1).sum(axis=1) - factor_ret_m['rf']) / (beta_m.mul(wL_m, axis=1).sum(axis=1).replace(0, np.nan))
exc_ret_bab_m = exc_ret_bab_L_m - exc_ret_bab_H_m

In [5]:
### Printing the sharpe ratios
print(f"The sharpe ratios of BAB with monthly rebalancing are {exc_ret_bab_m.mean() / exc_ret_bab_m.std() * np.sqrt(12):.2f}")
print(f"The sharpe ratios of BAB High with monthly rebalancing are {exc_ret_bab_H_m.mean() / exc_ret_bab_H_m.std() * np.sqrt(12):.2f}")
print(f"The sharpe ratios of BAB Low with monthly rebalancing are {exc_ret_bab_L_m.mean() / exc_ret_bab_L_m.std() * np.sqrt(12):.2f}")

The sharpe ratios of BAB with monthly rebalancing are 0.80
The sharpe ratios of BAB High with monthly rebalancing are 0.34
The sharpe ratios of BAB Low with monthly rebalancing are 0.92


In [6]:
### Using monthly weights to get the daily returns
wH_d = pd.DataFrame(index=stock_ret_d.index, columns=stock_ret_d.columns)
wL_d = pd.DataFrame(index=stock_ret_d.index, columns=stock_ret_d.columns)
beta_d = pd.DataFrame(index=stock_ret_d.index, columns=stock_ret_d.columns)

for date in w_m.index:
    short_date = date.strftime('%Y-%m')
    wH_d.loc[short_date] = wH_m.loc[date].values
    wL_d.loc[short_date] = wL_m.loc[date].values
    beta_d.loc[short_date] = beta_m.loc[date].values

### Getting the daily portfolio returns
exc_ret_bab_H_d = (stock_ret_d.mul(wH_d, axis=1).sum(axis=1) - factor_ret_d['rf']) / (beta_d.mul(wH_d, axis=1).sum(axis=1).replace(0, np.nan))
exc_ret_bab_L_d = (stock_ret_d.mul(wL_d, axis=1).sum(axis=1) - factor_ret_d['rf']) / (beta_d.mul(wL_d, axis=1).sum(axis=1).replace(0, np.nan))
exc_ret_bab_d = exc_ret_bab_L_d - exc_ret_bab_H_d

In [7]:
### Saving the data
conn = sqlite3.connect('Data/data.db')

factor_ret_m['BAB'] = exc_ret_bab_m
factor_ret_d['BAB'] = exc_ret_bab_d

factor_ret_d.to_sql('factors_daily', conn, if_exists='replace')
factor_ret_m.to_sql('factors_monthly', conn, if_exists='replace')

conn.close()